In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from time import sleep
import json

model_path = r'guru.h5'
try:
    model = load_model(model_path)
except Exception as e:
    print("Error loading the model:", e)
    exit(1)

classes = ['coco', 'coonut']
timer = 5
prev_detected_class = None

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Failed to open camera")
    exit(1)

# Create a placeholder for display_frame outside the loop
display_frame = None

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error: Failed to capture frame")
        break

    timer -= 1

    if timer == 0:
        timer = 5

        # Create a copy of the frame for display
        display_frame = frame.copy()

        frame = cv2.resize(frame, (224, 224))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = frame / 255.0
        frame = np.expand_dims(frame, axis=0)

        try:
            predictions = model.predict(frame)
            predicted_class_index = np.argmax(predictions)
            predicted_class = classes[predicted_class_index]

            if predicted_class != prev_detected_class:
                if predicted_class in ['coco', 'coonut']:
                    print(json.dumps({"Detected": predicted_class}))
                else:
                    print("Detected:", predicted_class)
                prev_detected_class = predicted_class
        except Exception as e:
            print("Error during prediction:", e)

    # Display the copy of the frame if it's defined
    if display_frame is not None and display_frame.shape[0] > 0 and display_frame.shape[1] > 0 and display_frame.shape[2] == 3:
        cv2.imshow('Frame', display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close the window
cap.release()
try:
    cv2.destroyAllWindows()
except Exception as e:
    print("Error occurred while closing the OpenCV windows:", e)


Error: Failed to capture frame
Error occurred while closing the OpenCV windows: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'



In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping

# Path to your dataset
dataset_path = r"D:\projects\coconut\coco-detect\data"

# Assuming your dataset is organized in folders with class names
classes = ['coco', 'coonut']

# Load dataset
def load_dataset():
    X, y = [], []
    for i, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, 'train', class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            try:
                image = cv2.imread(image_path)
                if image is not None:
                    image = cv2.resize(image, (224, 224))
                    X.append(image)
                    y.append(i)  # Label for the class
                else:
                    print(f"Failed to load image: {image_path}")
            except Exception as e:
                print(f"Error loading image {image_path}: {str(e)}")
    X = np.array(X)
    y = np.array(y)
    return X, y

# Load and preprocess dataset
X, y = load_dataset()

# Split dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Create data generators with increased data augmentation and noise for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.7, 1.3],  # Adjust brightness randomly
    preprocessing_function=lambda x: x + np.random.normal(0, 0.1, x.shape)  # Add random noise
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, y_val, batch_size=32)

# Create test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow(X_test, y_test, batch_size=32, shuffle=False)

# Use transfer learning with MobileNetV2 as the base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head with additional regularization techniques
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(len(classes), activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile model with additional regularization
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Learning rate schedule function
def lr_schedule(epoch):
    return 0.001 * (0.9 ** epoch)

# Learning rate scheduler
lr_scheduler = LearningRateScheduler(lr_schedule)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model with learning rate scheduling, early stopping, and additional regularization
model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[lr_scheduler, early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Save the trained model
model.save('guru.h5')





Epoch 1/50


64/64 [==============================] - 60s 791ms/step - loss: 0.2925 - accuracy: 0.9009 - val_loss: 0.1086 - val_accuracy: 0.9688 - lr: 0.0010
Epoch 2/50
64/64 [==============================] - 52s 805ms/step - loss: 0.0586 - accuracy: 0.9868 - val_loss: 0.0335 - val_accuracy: 0.9922 - lr: 9.0000e-04
Epoch 3/50
64/64 [==============================] - 56s 866ms/step - loss: 0.0382 - accuracy: 0.9941 - val_loss: 0.0224 - val_accuracy: 0.9922 - lr: 8.1000e-04
Epoch 4/50
64/64 [==============================] - 53s 828ms/step - loss: 0.0189 - accuracy: 0.9951 - val_loss: 0.0053 - val_accuracy: 1.0000 - lr: 7.2900e-04
Epoch 5/50
64/64 [==============================] - 52s 810ms/step - loss: 0.0235 - accuracy: 0.9961 - val_loss: 0.0050 - val_accuracy: 1.0000 - lr: 6.5610e-04
Epoch 6/50
64/64 [==============================] - 50s 781ms/step - loss: 0.0091 - accuracy: 0.9980 - val_loss: 0.0044 - val_accuracy: 0.9961 - lr: 5.9049e-04
Epoch 7/50
64/64 [=====================

c:\Users\Thenmozhi\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
